# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model

In [ ]:
!wget -O train.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/test.txt

### 1.1 Building vocabulary

#### Code

In [19]:
from itertools import count
import numpy as np

# So, build the model based on make the vocabulary set on training set
f = open("./data/lm/train.txt",'r+',encoding="utf-8")
lines = []
s = set(["<s>","</s>"])
sentence = []
for line in f.readlines()[:-1]:
    # some lines are still end with \n, need to remove \n
    if line.endswith("\n"):
        line = line[:-1]
    tmp = line.split(' ')
    sentence.append(tmp)
    for word in tmp:
        s.add(word)

word_list = sorted(list(s))

word_count = np.zeros(len(word_list),dtype=int)

word_dic = {each:idx for (idx,each) in enumerate(word_list)}

startC = word_dic["<s>"]
endC = word_dic["</s>"]

# count every word
for sen in sentence:
    word_count[startC] += 1
    word_count[endC] += 1
    for word in sen:
        i = word_dic[word]
        word_count[i]+=1

#vocabulary size
print("vocabulary size: %d"%len(word_list))


# the word_dic : the map between words and id, 
# word_count : the number of each word organized with id
# sentence : sentences in training data

vocabulary size: 52642


#### Discussion

Please show the vocabulary size and discuss the number of parameters of n-gram models.

The vocabulary size is 52,642, including <s> and </s> characters. So, as for the number of parameters of n-gram models, when the n increases, the number of parameters increases sharply. For example, when n is 1 in this case, the number of parameters is 52642 which is the vocabulary size. When n is 2, the vocabulary size is 2,771,180,164 which is the square of vocabulary size. For n equals 3, the vocabulary size is the cube of vocabulary size :145,880,466,193,288. If we use a float variable to store a single probability, it would use 543,447.085444957GB memory.

### 1.2 $n$-gram Language Modeling

#### Code

#### Discussion

### 1.3 Smoothing

#### 1.3.1 Add-one (Laplace) smoothing

##### Code

##### Discussion

##### Optional: Add-k smoothing

###### Code

###### Discussion

#### 1.3.2 Linear Interpolation

##### Code

##### Discussion

##### Optional: Optimization

###### Discussion

## 2 Preposition Prediction

In [ ]:
!wget -O dev.in https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.out